アンサンブル
https://www.loom.com/share/13abc0b6b3ae4258a15527dbb2bd5924
https://www.loom.com/share/3a8639fadbaf4e59a191c8d28f8d2867
https://www.loom.com/share/6215e2af936d4d09be2fd741a1a6fc54
https://www.loom.com/share/7ff037e5eb344b70bbc250578334cd67

In [66]:
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from sklearn.metrics import confusion_matrix
from pandas.plotting import scatter_matrix
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import scipy.stats as stats
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
%matplotlib inline

住宅価格の予想からデータを引っ張って来る
---
参考URL:https://qiita.com/Moby-Dick/items/e2f1efd923fddd72b17d


In [67]:
df = pd.read_csv('train.csv')

>目的変数をSalePrice, 説明変数をGrLivAreaとYearBuilt

In [68]:
X = df.loc[:, ['GrLivArea', 'YearBuilt']]
y = df.loc[:,['SalePrice']]
print(X.shape)
print(y.shape)

(1460, 2)
(1460, 1)


In [69]:
X.describe()

,GrLivArea,YearBuilt
count,1460.000000,1460.000000
mean,1515.463699,1971.267808
std,525.480383,30.202904
min,334.000000,1872.000000
25%,1129.500000,1954.000000
50%,1464.000000,1973.000000
75%,1776.750000,2000.000000
max,5642.000000,2010.000000


In [70]:
y.describe()

,SalePrice
count,1460.000000
mean,180921.195890
std,79442.502883
min,34900.000000
25%,129975.000000
50%,163000.000000
75%,214000.000000
max,755000.000000


【問題1】ブレンディングのスクラッチ実装
----
>ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。

In [71]:
clf1 = lgb.LGBMRegressor()
clf2 = DecisionTreeRegressor(max_depth=2)
clf3 = ElasticNet()

In [72]:
num_test = 0.20
sc = StandardScaler()
X = np.array(X)
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=num_test, random_state=23)
sc.fit(X_train)
X_train=sc.transform(X_train)

In [73]:
clf1.fit(X_train,y_train) #LGBM
y_pred1 = clf1.predict(X_test)
np.log(mean_squared_error(y_test, y_pred1))

/Users/harakenken22/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


24.996542292830945

In [74]:
clf2.fit(X_train,y_train) #決定木
y_pred2 = clf2.predict(X_test)
np.log(mean_squared_error(y_test, y_pred2))

23.852516529559665

In [75]:
clf3.fit(X_train,y_train) #ElasticNet
y_pred3 = clf3.predict(X_test)
np.log(mean_squared_error(y_test, y_pred3))

36.81102418017892

In [76]:
y_pred_all = (y_pred1 + y_pred2 + y_pred3)/3 #単純に三つの回帰を足している
np.log(mean_squared_error(y_test, y_pred_all))

34.62129087814806

【問題2】バギングのスクラッチ実装
----
バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

In [77]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0)

models = {
    'not bagging': DecisionTreeClassifier(random_state=0),
    'bagging': BaggingClassifier(DecisionTreeClassifier(random_state=0), n_estimators=100, random_state=0) 
}

scores = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    scores[(model_name, 'train_score')] = model.score(X_train, y_train)
    scores[(model_name, 'test_score')] = model.score(X_test, y_test)

pd.Series(scores).unstack()

/Users/harakenken22/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


,test_score,train_score
bagging,0.008219,0.957078
not bagging,0.005479,0.957078


【問題3】スタッキングのスクラッチ実装
---
スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

In [79]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
X_train_stack, X_test_stack, y_train_stack, y_test_stack = train_test_split(
    X, y)

# train base model
base_model_1 = LinearRegression()
base_model_2 = lgb.LGBMRegressor()
base_model_3 = KNeighborsRegressor()

base_model_1.fit(X_train_stack, y_train_stack)
base_model_2.fit(X_train_stack, y_train_stack)
base_model_3.fit(X_train_stack, y_train_stack)

# base predicts
base_pred_1 = base_model_1.predict(X_test_stack)
base_pred_2 = base_model_2.predict(X_test_stack)
base_pred_3 = base_model_3.predict(X_test_stack)

/Users/harakenken22/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


メタモデルの学習
----
ベースモデルの出力を使ってメタモデルの学習を行います。 そんなに難しいことはしておらず、「ベースモデルの出力を連結する」「それを入力として学習させる」だけです。

np.column_stack
---
np.column_stack((行列1, 行列2)) : １次行列の組み合わせなら２次元行列を作り、多次元の組みならhstackと同じ振る舞いをする
>https://qiita.com/gragragrao/items/1881accf27bbedc5fa84

In [80]:
# stack base predicts for training meta model
stacked_predictions = np.column_stack((base_pred_1, base_pred_2, base_pred_3))

In [82]:
# train meta model 
meta_model = LinearRegression()
meta_model.fit(stacked_predictions, y_test_stack)

LinearRegression()

In [90]:
# test predicts for final result 
valid_pred_1 = base_model_1.predict(X_test_stack)
valid_pred_2 = base_model_2.predict(X_test_stack)
valid_pred_3 = base_model_3.predict(X_test_stack)
stacked_valid_predictions = np.column_stack((valid_pred_1, valid_pred_2, valid_pred_3))
meta_valid_pred = meta_model.predict(stacked_valid_predictions)

print ("mean squared error of model 1: {:.4f}".format(mean_squared_error(y_test_stack, valid_pred_1)))
print ("mean squared error of model 2: {:.4f}".format(mean_squared_error(y_test_stack, valid_pred_2)))
print ("mean squared error of model 3: {:.4f}".format(mean_squared_error(y_test_stack, valid_pred_3)))
print ("mean squared error of meta model: {:.4f}".format(mean_squared_error(y_test_stack, meta_valid_pred)))

mean squared error of model 1: 2760059189.1690
mean squared error of model 2: 2256653644.6304
mean squared error of model 3: 3202532773.2064
mean squared error of meta model: 2148930974.3305
